In [1]:
import h5py
import random
import numpy as np
import tensorflow as tf
from datetime import datetime
import H5pyHelper
from tensorflow.keras import layers, models, regularizers, optimizers
import dask.array as da
from dask.array.slicing import shuffle_slice
import cv2
#from sklearn.metrics import accuracy_score

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

hf = h5py.File('/workspaces/flora_dex/h5_files/copies/data.h5', 'r')
BATCH_SIZE = 32
NUM_CLASSES = hf['y_train'][0].shape[0] ## gets number of classes
INP_SHAPE = hf['x_train'][0].shape
TRAIN_SIZE = hf["x_train"].shape[0]
TEST_SIZE = hf["x_test"].shape[0]

lol = hf["x_train"]
x = da.from_array(lol,chunks=(100,500,500,3))
#x = da.from_array(lol)

print (lol[0])

np.random.seed(42)
index = np.random.choice(TRAIN_SIZE,TRAIN_SIZE,replace=False)
print (index)
d_arry = shuffle_slice(x,index)

print (d_arry)

#x_new = d_arry.compute()

#print (x_new[0])


hf.close()

pics = []
for i in range(1):
    for j in range(32):
        pics.append(cv2.imread(f"/workspaces/flora_dex/raw_data/Abronia elliptica/{j}.jpg"))
    print(i)



        

Num GPUs Available:  0
[[[ 0.37635422  0.4665503   0.29400128]
  [ 0.38005617  0.46284834  0.29770324]
  [ 0.38137382  0.45541304  0.29666793]
  ...
  [-0.145685   -0.12215561 -0.1306262 ]
  [-0.19227324 -0.19227324 -0.19227324]
  [-0.19227324 -0.19227324 -0.19227324]]

 [[ 0.3689503   0.47025225  0.29029933]
  [ 0.37090492  0.46567664  0.29487494]
  [ 0.37160066  0.4583746   0.29548332]
  ...
  [ 0.01164832  0.13335373  0.08927029]
  [-0.10752796 -0.05510819 -0.07432877]
  [-0.19227324 -0.19227324 -0.19227324]]

 [[ 0.35972676  0.474864    0.2885111 ]
  [ 0.35994887  0.46931106  0.2927313 ]
  [ 0.3604797   0.4622146   0.29419577]
  ...
  [ 0.14725304  0.35416126  0.2793503 ]
  [-0.03427355  0.0634582   0.02762323]
  [-0.19227324 -0.19227324 -0.19227324]]

 ...

 [[-0.03603794 -0.07619481 -0.09188109]
  [-0.03137347 -0.07382555 -0.09091858]
  [-0.02751089 -0.07194697 -0.09014305]
  ...
  [ 0.37748364  0.50570947  0.30167502]
  [ 0.38046905  0.51076174  0.30225477]
  [ 0.38372678  0.515

In [23]:

base = []


for i in range(100):
    l = np.ones((500,500,3)) * i
    base.append(l)

base = np.array(base)

#print (base)


#x = da.from_array(base ,chunks= (20,500,500,3))
x = da.from_array(base)

index = np.random.choice(100,100,replace=False)

print(index)
print (x)
d_arry = shuffle_slice(x,index)

x_new = d_arry.compute()

print (x_new[0])



[56 66 31  7 55 61 65 84 54 28 23 12 35 37 59 62 16 91 10 98 95 90 13 60
 29 87 85 43 69  6  0 89 34  1 94 14 17 93 63 48 49 86 68 32 82 51 58  4
  5  2 39 15  8 27 57 11 76 50 30 22 99 26 78 80 47 97 71 74 46 52 75 44
 64 96 41 40 67 25 38 70 53 83 45  9 18  3 36 73 20 72 88 21 81 79 19 42
 24 77 92 33]
dask.array<array, shape=(100, 500, 500, 3), dtype=float64, chunksize=(100, 125, 125, 3), chunktype=numpy.ndarray>
[[[56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]
  ...
  [56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]]

 [[56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]
  ...
  [56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]]

 [[56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]
  ...
  [56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]]

 ...

 [[56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]
  ...
  [56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]]

 [[56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]
  ...
  [56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]]

 [[56. 56. 56.]
  [56. 56. 56.]
  [56. 56. 56.]
  ...


In [68]:
# A memory-mapped array is kept on disk. However, it can be accessed and sliced like any ndarray.
# Memory mapping is especially useful for accessing small fragments of large files without reading
# the entire file into memory.

#def generator(feature_name,label_name,batch_size,shuffle):
#    index = 0
#    
#    hf = h5py.File('/home/sorozco0612/dev/flora_dex/raw_data/data.h5', 'a')
#    
#    # shuffle data
#    if (shuffle):
#        print("Shuffling data before starting training...")
#        #random.seed(datetime.now())
#
#        #random.shuffle(hf[feature_name])
#        #random.shuffle(hf[label_name])
#    
#    while True:
#        if index == 0:
#    
#            x = hf[feature_name]
#            y = hf[label_name] 
#            
#            ## create shuffle index for each batch
#            idx_map = np.arange(x.shape[0])
#            np.random.shuffle(idx_map)
#        
#        # batch has not met the end of the data
#        if (index + batch_size < x.shape[0]):
#            batch = sorted(idx_map[index:index+batch_size])
#            features = x[batch]
#            labels = y[batch]
np.append(base,x)
#
#            index += batch_size
#        else:
#            # batch size will be smaller than the rest on last iteration 
#            
#            batch = sorted(idx_map[index:])
#            features = x[batch]
#            labels = y[batch]
#            
#            idx_map = np.arange(features.shape[0])
#            np.random.shuffle(idx_map)
#
#            index = 0
#            
#            ## close file so it can be reshuffled
#            hf.close()
#        
#        ## shuffle just the batches
#        #features = features[idx_map]
#        #labels = labels[idx_map]
#        
#        yield (features,labels)
        
        
def generator(feature_name,label_name,batch_size,shuffle):
    index = 0
    
    
    
    if (shuffle):
        pass
        #print("Shuffling data before starting training...")
        #seed = datetime.now()
        
        #H5pyHelper.shuffle_dataset(
        #            "/home/sorozco0612/dev/flora_dex/raw_data/data.h5", feature_name, seed
        #        )
        #H5pyHelper.shuffle_dataset(
        #           "/home/sorozco0612/dev/flora_dex/raw_data/data.h5", label_name, seed
        #        )
    while True:
        features = []
        if index == 0:
            hf = h5py.File('/workspaces/flora_dex/raw_data/data_segmented.h5', 'r')
    
            x = hf[feature_name]
            y = hf[label_name]  
        
        # batch has not met the end of the data
        if (index + batch_size < x.shape[0]):
            for j in range(32):
                img = cv2.imread(f"/workspaces/flora_dex/raw_data/Abronia elliptica/{j}.jpg")
                img = cv2.resize(img,(500,500))
                features.append(img)
            #features = x[index:index+batch_size]
            labels = y[index:index+batch_size]
            index += batch_size
        else:
            # batch size will be smaller than the rest on last iteration
            features = x[index:]
            labels = y[index:]
            index = 0
            
            ## close file so it can be reshuffled
            hf.close()
        
        features = np.asarray(features)
            
        yield (features,labels)   

train_generator = generator('x_train', 'y_train',BATCH_SIZE,True)
test_generator = generator('x_test','y_test',BATCH_SIZE,False)

In [69]:
## alex net
model = models.Sequential([
    layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(500,500,3)),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 123, 123, 96)      34944     
_________________________________________________________________
batch_normalization_45 (Batc (None, 123, 123, 96)      384       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 61, 61, 96)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 61, 61, 256)       614656    
_________________________________________________________________
batch_normalization_46 (Batc (None, 61, 61, 256)       1024      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 30, 30, 384)      

In [70]:
#model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])

In [71]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
fit = model.fit(x=train_generator,
                steps_per_epoch= TRAIN_SIZE // BATCH_SIZE,
                callbacks=[callback], 
                validation_steps= TEST_SIZE // BATCH_SIZE,
                validation_data=test_generator,
                verbose=1,
                epochs=250)

Epoch 1/250
 32/768 [>.............................] - ETA: 41:44 - loss: 3.1871 - accuracy: 0.5566

KeyboardInterrupt: 

In [ ]:
def summarize_prediction(Y_true, Y_pred):
    #mse = mean_squared_error(Y_true, Y_pred)
    #accuracy = accuracy_score(Y_true, Y_pred)
    #print(f'mse       = {mse:.2}')
    print(f'accuracy = {accuracy:.2%}')
    
def predict_and_summarize(X, Y):
    Y_pred = model.predict(X)
    #summarize_prediction(Y, Y_pred)
    return Y_pred

hf = h5py.File('/home/sorozco0612/dev/flora_dex/raw_data/data.h5', 'r')
x_test = hf["x_test"]
y_test = hf["y_test"]

y_pred = predict_and_summarize(x_test, y_test)

print(y_pred[:1])
print(y_test[:1])

hf.close()